In [1]:
# default_exp model

# Model
> AdaptNLP's internal `AdaptiveModel` class, along with `fastai` internals for predicting

In [2]:
#hide
from nbverbose.showdoc import *

In [3]:
#export
from typing import Union, List
from pathlib import Path
from abc import ABC, abstractmethod

from flair.data import Sentence

import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader

from fastcore.basics import noop, store_attr, patch, first, ifnone
from fastcore.meta import delegates

from fastai.callback.core import Callback, GatherPredsCallback
from fastai.callback.progress import ProgressCallback

from fastai.learner import Learner
from fastai.data.core import DataLoaders

from fastai.torch_core import to_device, default_device

from adaptnlp.callback import GatherInputsCallback, SetInputsCallback

## Patched Functionalities

In [4]:
#export
@patch
def one_batch(
    self:DataLoader
) -> torch.Tensor: # A batch of data
    "Pathed functionality that grabs one batch of data from the `DataLoader` and deletes the iter"
    res = first(self)
    if hasattr(self, 'it'): delattr(self, 'it')
    return res

In [5]:
#export
@patch
def after_validate(
    self:GatherPredsCallback
):
    "Patched functionality that does nothing"
    pass

In [6]:
#export
class CudaCallback(Callback):
    "Move data to CUDA device"
    def __init__(
        self, 
        device:str=None # A device to move the data to, such as 'cuda:0' or 'cpu'
    ): self.device = ifnone(device, default_device())
    def before_batch(self): self.learn.xb,self.learn.yb = to_device(self.xb),to_device(self.yb)
    def before_fit(self): self.model.to(self.device)

In [7]:
#exporti
class _NoopModel(nn.Module):
    """
    Very basic model that performs y = mx+b. Is used only as a placeholder model for `BaseLearner`.
    Is based on https://github.com/fastai/fastai/blob/master/fastai/test_utils.py#L30
    """
    def __init__(self):
        super().__init__()
        self.a,self.b = nn.Parameter(torch.randn(1)),nn.Parameter(torch.randn(1))

    def forward(self, x): return x*self.a + self.b

In [8]:
#exporti
class _BaseLearner:
    """
    Simple `Learner` class with `synth` DataLoaders, a noop model, and a noop loss function.

    Contains access to minimal `Learner` functionality including:
      - `get_preds`
      - `lr_find`, `fit_one_cycle`, `fit_flat_cos`, `fit_sgdr`, `fit` (not implemented)
      - `metrics`, `opt_func`, `splitter`, `wd`, `moms` (not implemented)
    """
    __cbs = [SetInputsCallback(), GatherInputsCallback()]

    def __init__(self, device='cuda' if torch.cuda.is_available() else 'cpu') -> None:
        """
        Generates blank `Learner` and stores it away privately.
        """
        self.__cbs.append(CudaCallback(device))
        self.__learner = Learner(self._generate_dls(), _NoopModel(), loss_func=noop, cbs=self.__cbs)
        self.__default_dls, self.__default_model = True, True

    def _generate_dls(self, a=2, b=3, batch_size=16, n_train=10, n_valid=2) -> DataLoaders:
        """
        Builds synthetic `DataLoaders`.
        Implementation based on https://github.com/fastai/fastai/blob/master/fastai/test_utils.py#L18
        """
        def get_data(n) -> TensorDataset:
            """
            Generates synthetic `TensorDataset`
            """
            x = torch.randn(batch_size*n, 1)
            return TensorDataset(x, a*x + b, 0.1*torch.randn(batch_size*n, 1))

        train_ds = get_data(n_train)
        valid_ds = get_data(n_valid)
        train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=0)
        valid_dl = DataLoader(valid_ds, batch_size=batch_size, num_workers=0)
        return DataLoaders(train_dl, valid_dl)

    def get_preds(self, dl=None, cbs=[]):
        """
        Get raw predictions based on `dl` with `cbs`.

        For basic inference, `cbs` should include any `Callbacks` needed to do general inference
        """
        if dl is None: raise ValueError("`dl` should not be `None`")
        if isinstance(self.__learner.model, _NoopModel):
            raise ValueError("The default model is still set, you should override this with `_BaseLearner.set_model(x)`")
        return self.__learner.get_preds(dl=dl, cbs=cbs)
    
    def set_device(self, device:str='cpu'):
        if device != 'cpu' and device != 'cuda': 
            raise ValueError("Device must either be `cpu` or `cuda`")
        self.__learn.__cbs[-1].device = device

    def set_as_dict(self, as_dict:bool=False):
        """
        Sets `is_dict` in the `SetInputsCallback`. Should be utilized whenever dictating
        if the incoming batch should be a dictionary or the actual inputs.
        """
        for cb in self.__cbs:
            if isinstance(cb, SetInputsCallback):
                cb.as_dict = as_dict

    def set_model(self, model):
        """
        Sets `Learner`'s model to `model`
        """
        self.__learner.model = model

In [9]:
#export
class AdaptiveModel(ABC):
    _learn = _BaseLearner()
    
    def set_model(
        self, 
        model # A PyTorch model
    ):
        "Sets model in `_learn`"
        self._learn.set_model(model)
        self.model = model
        
    def set_as_dict(
        self, 
        as_dict:bool=False # Whether to return the inputs as a dictionary when predicting or training
    ):
        "Sets `as_dict` in `_learn`"
        self._learn.set_as_dict(as_dict)
        
    def set_device(
        self, 
        device:str='cpu' #  A device for the `CudaCallback`, such as 'cuda:0' or 'cpu'
    ):
        "Sets the device for `CudaCallback` in `__learn`"
        self._learn.set_device(device)
        

    def get_preds(
        self, 
        dl=None, # An iterable DataLoader or DataLoader-like object 
        cbs=[] # Optional fastai `Callbacks`
    ):
        """
        Get raw predictions based on `dl` with `cbs`.

        For basic inference, `cbs` should include any `Callbacks` needed to do general inference
        """
        return self._learn.get_preds(dl=dl, cbs=cbs)
        
    @abstractmethod
    def load(
        self,
        model_name_or_path: Union[str, Path], # A model file location to use
    ):
        "Load model into the `AdaptiveModel` object as alternative constructor"
        raise NotImplementedError("Please Implement this method")

    @abstractmethod
    def predict(
        self,
        text: Union[List[Sentence], Sentence, List[str], str], # Some text to predict on
        mini_batch_size: int = 32, # A batch size for if a list of texts are passed in
        **kwargs,
    ) -> List[Sentence]: # A list of predicted sentences
        "Run inference on the model"
        raise NotImplementedError("Please Implement this method")

In [10]:
show_doc(AdaptiveModel.set_model)

<h4 id="AdaptiveModel.set_model" class="doc_header"><code>AdaptiveModel.set_model</code><a href="__main__.py#L5" class="source_link" style="float:right">[source]</a></h4>

> <code>AdaptiveModel.set_model</code>(**`model`**)

Sets model in `_learn`

**Parameters:**


 - **`model`** : *`<class 'inspect._empty'>`*	<p>A PyTorch model</p>



In [11]:
show_doc(AdaptiveModel.set_as_dict)

<h4 id="AdaptiveModel.set_as_dict" class="doc_header"><code>AdaptiveModel.set_as_dict</code><a href="__main__.py#L13" class="source_link" style="float:right">[source]</a></h4>

> <code>AdaptiveModel.set_as_dict</code>(**`as_dict`**:`bool`=*`False`*)

Sets `as_dict` in `_learn`

**Parameters:**


 - **`as_dict`** : *`<class 'bool'>`*, *optional*	<p>Whether to return the inputs as a dictionary when predicting or training</p>



In [12]:
show_doc(AdaptiveModel.set_device)

<h4 id="AdaptiveModel.set_device" class="doc_header"><code>AdaptiveModel.set_device</code><a href="__main__.py#L20" class="source_link" style="float:right">[source]</a></h4>

> <code>AdaptiveModel.set_device</code>(**`device`**:`str`=*`'cpu'`*)

Sets the device for [`CudaCallback`](/adaptnlp/model.html#CudaCallback) in `__learn`

**Parameters:**


 - **`device`** : *`<class 'str'>`*, *optional*	<p>A device for the `CudaCallback`, such as 'cuda:0' or 'cpu'</p>



In [13]:
show_doc(AdaptiveModel.get_preds)

<h4 id="AdaptiveModel.get_preds" class="doc_header"><code>AdaptiveModel.get_preds</code><a href="__main__.py#L28" class="source_link" style="float:right">[source]</a></h4>

> <code>AdaptiveModel.get_preds</code>(**`dl`**=*`None`*, **`cbs`**=*`[]`*)

Get raw predictions based on `dl` with `cbs`.

For basic inference, `cbs` should include any `Callbacks` needed to do general inference

**Parameters:**


 - **`dl`** : *`<class 'NoneType'>`*, *optional*	<p>An iterable DataLoader or DataLoader-like object</p>


 - **`cbs`** : *`<class 'list'>`*, *optional*	<p>Optional fastai `Callbacks`</p>



In [14]:
show_doc(AdaptiveModel.load)

<h4 id="AdaptiveModel.load" class="doc_header"><code>AdaptiveModel.load</code><a href="__main__.py#L40" class="source_link" style="float:right">[source]</a></h4>

> <code>AdaptiveModel.load</code>(**`model_name_or_path`**:`Union`\[`str`, `Path`\])

Load model into the [`AdaptiveModel`](/adaptnlp/model.html#AdaptiveModel) object as alternative constructor

**Parameters:**


 - **`model_name_or_path`** : *`typing.Union[str, pathlib.Path]`*	<p>A model file location to use</p>



In [15]:
show_doc(AdaptiveModel.predict)

<h4 id="AdaptiveModel.predict" class="doc_header"><code>AdaptiveModel.predict</code><a href="__main__.py#L48" class="source_link" style="float:right">[source]</a></h4>

> <code>AdaptiveModel.predict</code>(**`text`**:`Union`\[`List`\[`Sentence`\], `Sentence`, `List`\[`str`\], `str`\], **`mini_batch_size`**:`int`=*`32`*, **\*\*`kwargs`**)

Run inference on the model

**Parameters:**


 - **`text`** : *`typing.Union[typing.List[flair.data.Sentence], flair.data.Sentence, typing.List[str], str]`*	<p>Some text to predict on</p>


 - **`mini_batch_size`** : *`<class 'int'>`*, *optional*	<p>A batch size for if a list of texts are passed in</p>


 - **`kwargs`** : *`<class 'inspect._empty'>`*


**Returns**:
	
 * *`typing.List[flair.data.Sentence]`*	<p>A list of predicted sentences</p>

